In [1]:
import numpy as np
from PIL import Image
import os
import cv2
import torch 
import torch.nn as nn 
from torch.utils.data import DataLoader, Dataset, random_split, Subset
import torchvision
from torchvision.io import read_image
from torchvision import datasets, models, transforms
from tqdm import tqdm
import regex as re 
import pandas as pd
import os
import wandb

In [2]:
# # run this cell if running the notebook for the first time. 
def sort_fn(x):
    num1 = re.findall(r"image([\d]+)", x)
    return int(num1[0])

# Sort read in image filenames. 
image_dir = "./images/"
image_filenames = os.listdir(os.path.join(os.getcwd(), "images"))
image_filenames.sort(key= sort_fn)

# Fetch complete image dir. 
image_filenames = [os.path.join(image_dir, i) for i in image_filenames]
len(image_filenames)

labels = [1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1 ,1,1,1,0, 1, 0, 0, 0 ,1,0,1, 0, 0, 1, 0, 0, 1, 1,0, 1, 0,0,1, 1, 1, 1, 1, 1, 1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,1,0,1,1,0,0,1,1,1,1,1,0,0,1,1,0,1,1,1,1,0,0,0,1,1,0,1]
pd.DataFrame({"image":image_filenames, "label":labels}).to_csv("train_data.csv", index=False)

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [3]:
class WindmillDataset(Dataset):
    def __init__(self, transform=None):
        self.data_dir = 'windmill'
        valid_targets = os.listdir(os.path.join(self.data_dir, 'val', 'with_mill'))
        valid_backgrounds = os.listdir(os.path.join(self.data_dir, 'val', 'without'))
        self.valid_file_paths = valid_targets + valid_backgrounds

        train_targets = os.listdir(os.path.join(self.data_dir, 'train', 'with_mill'))
        train_backgrounds = os.listdir(os.path.join(self.data_dir, 'train', 'without'))
        self.train_file_paths = train_targets + train_backgrounds

        valid_data = pd.concat((pd.DataFrame({'name': valid_targets, 'label': 1}),
                             pd.DataFrame({'name': valid_backgrounds, 'label': 0})))
        self.valid_labels = torch.from_numpy(valid_data.iloc[:, 1].to_numpy()).to(torch.long)

        train_data = pd.concat((pd.DataFrame({'name': train_targets, 'label': 1}),
                             pd.DataFrame({'name': train_backgrounds, 'label': 0})))
        self.train_labels = torch.from_numpy(train_data.iloc[:, 1].to_numpy()).to(torch.long)

    def __len__(self):
        return len(self.train_file_paths) + len(self.valid_file_paths)

    def __getitem__(self, idx):
    # idx < 770 is validation data, others are train
        if idx < 770:
            img_path = self.valid_file_paths[idx]
            label = self.valid_labels[idx]
            full_path = os.path.join(os.path.join(self.data_dir, 'val', ('with_mill' if label else 'without')), img_path)
        else:
            real_id = idx - 770
            img_path = self.train_file_paths[real_id]
            label = self.train_labels[real_id]
            full_path = os.path.join(os.path.join(self.data_dir, 'train', ('with_mill' if label else 'without')), img_path)

        image = read_image(full_path)
        return image, label


In [4]:
class ImageDataset(Dataset):
    # Set output = False if doesn't want the print. 
    def __init__(self, image_filenames, labels):
        super().__init__()
        self.image_filenames = image_filenames
        self.labels = labels
        # Read all images into numpy array. 
        images = []
        for i in self.image_filenames:
            images.append(cv2.imread(i))
        assert len(images) == len(self.image_filenames)
        assert len(images) == len(labels)
        # Convert to tensor and transpose. 
        self.images = torch.tensor(np.array(images)).transpose(1, -1)

        print("The shape of all images: ", self.images.shape)

    def __getitem__(self, i):
        final_image = self.images[i]
        return final_image, self.labels[i]

    def __len__(self):
        return len(self.image_filenames)

In [21]:
def load_checkpoint(model, save_name, save_dir = "model_ckpt"):
    save_path = os.path.join(save_dir, save_name)
    assert os.path.exists(save_path), "Path doesn't exist!"
    print("Loading checkpoint...")
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    best_loss = checkpoint['loss']
    print("Done!")

    return best_loss


def train(model, optimizer, data_augment, scheduler, train_dataloader, validation_dataloader, criterion, num_epochs = 5, load_from_save =False, save_name = "model0", save_dir = "model_ckpt"):
    best_acc = 0.0
    if load_from_save:
        print("Loading the model...")
        best_acc = load_checkpoint(model, optimizer, save_name + ".tar", save_dir)
        print("Done!")
    print(optimizer.param_groups[0]["lr"])
    save_path = os.path.join(save_dir, save_name + ".tar")
    best_state_dict = {}

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        num_correct = 0
        for _, (inputs, labels) in enumerate(tqdm(train_dataloader)):
            inputs = inputs.float().to(device)
            if data_augment:
                inputs = data_augment(inputs)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            pred = model(inputs)
            loss = criterion(pred, labels)
            num_correct += (pred.argmax(1) == labels).sum().item()
            loss.backward()
            optimizer.step()
            if scheduler:
                scheduler.step()
            train_loss += loss.item()
            del inputs, labels, pred
        
        epoch_loss = train_loss / len(train_dataloader.dataset)
        print(f"Epoch {epoch} training loss: ", epoch_loss)
        valid_loss, valid_acc = evaluate(model, validation_dataloader=validation_dataloader, criterion=criterion)
        wandb.log({'valid epoch loss': valid_loss,
                    'valid accuracy': valid_acc,
                    'train epoch loss': epoch_loss, 
                    'train epoch accuracy': num_correct / len(train_dataloader.dataset)})

        if valid_acc > best_acc:
            if not os.path.exists(save_dir):
                os.mkdir(save_dir)
            
            best_acc = valid_acc
            state_dict = {
                'model_state_dict': model.state_dict(),
                'optim_state_dict': optimizer.state_dict(),
                'loss': best_acc
            }
            best_state_dict = state_dict
            
            torch.save(state_dict, save_path)
            print(f"Saved checkpoint to {save_path}")

        print(f"Best valid accuracy: {best_acc}")
    model.load_state_dict(best_state_dict['model_state_dict'])

def evaluate(model, validation_dataloader, criterion):
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        correct = 0
        for _, (inputs, labels) in tqdm(enumerate(validation_dataloader), total=len(validation_dataloader)):
            inputs = inputs.float().to(device)
            labels = labels.to(device)
            pred = model(inputs)
            pred_labels = pred.argmax(1)
            correct += (pred_labels == labels).sum().item()
            loss = criterion(pred, labels)
            valid_loss += loss.item()
        acc = correct / len(validation_dataloader.dataset)
        valid_loss = valid_loss / len(validation_dataloader.dataset)
        print("Validation loss: ", valid_loss)
        print("Validation acc: ", acc)
        return valid_loss, acc

In [8]:
##### Windmill #####
batch_size = 8

ds = WindmillDataset()
valid_data, train_data = Subset(ds, torch.arange(0, 770)), Subset(ds, torch.arange(770, len(ds)))
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)

In [9]:
###### Finetune on windmill ######
lr = 1e-3
num_epochs = 6
model = models.efficientnet_b5(weights='DEFAULT')
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = None

data_augment = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
])

wandb.login()
wandb.init(project='182')
wandb.run.name = 'windmill_effnet'
model_save_name = wandb.run.name
train(model, optimizer=optimizer, data_augment=data_augment, scheduler=scheduler, train_dataloader=train_dataloader, validation_dataloader=validation_dataloader, criterion=criterion, num_epochs=num_epochs, save_name=model_save_name)
wandb.run.finish()

wandb: Currently logged in as: jonluj. Use `wandb login --relogin` to force relogin


0.001


100%|██████████| 388/388 [01:00<00:00,  6.40it/s]


Epoch 0 training loss:  0.024777112509492005


100%|██████████| 97/97 [00:03<00:00, 25.15it/s]


...

100%|██████████| 388/388 [00:55<00:00,  6.96it/s]


Epoch 5 training loss:  0.008209522694209402


100%|██████████| 97/97 [00:03<00:00, 25.72it/s]

Validation loss:  0.007476578584913574
Validation acc:  0.9805194805194806
Best valid accuracy: 0.987012987012987


train epoch accuracy,▁▆█▇██
train epoch loss,█▃▃▃▁▁
valid accuracy,▅▆█▁█▇
valid epoch loss,▄▃▂█▁▃
train epoch accuracy,0.97774
train epoch loss,0.00821
valid accuracy,0.98052
valid epoch loss,0.00748


In [22]:
##### Main Task #####
np.random.seed(42)
torch.manual_seed(42)
df = pd.read_csv("train_data1.csv")
data = df.sample(frac=1)

# Build the dataset. 
batch_size = 8

ds = ImageDataset(data['image'].values.tolist(), data['label'].values.tolist())
train_dataset, validation_dataset, test_dataset = random_split(ds, [0.65, 0.2, 0.15], generator=torch.Generator().manual_seed(42))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

The shape of all images:  torch.Size([406, 3, 500, 500])


In [12]:
##### Train Head #####
lr = 1e-3
num_epochs = 20

# Set all params besides the new head to be frozen
for param in model.parameters():
    param.requires_grad = False
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

data_augment = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
])

# data_augment = None
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(train_dataloader), epochs=num_epochs)
scheduler = None

wandb.init(project='182')
wandb.run.name = 'effnet_head'
model_save_name = wandb.run.name
train(model, optimizer=optimizer, data_augment=data_augment, scheduler=scheduler, train_dataloader=train_dataloader, validation_dataloader=validation_dataloader, criterion=criterion, num_epochs=num_epochs, save_name=model_save_name)
wandb.run.finish()

0.001


100%|██████████| 33/33 [00:05<00:00,  6.51it/s]


Epoch 0 training loss:  0.08587763210137685


100%|██████████| 11/11 [00:01<00:00, 10.88it/s]


Validation loss:  0.09271145157697724
Validation acc:  0.524390243902439
Saved checkpoint to model_ckpt/effnet_head.tar
Best valid accuracy: 0.524390243902439


...

100%|██████████| 33/33 [00:04<00:00,  7.35it/s]


Epoch 19 training loss:  0.0843190923333168


100%|██████████| 11/11 [00:00<00:00, 13.10it/s]


Validation loss:  0.0907849495003863
Validation acc:  0.6219512195121951
Best valid accuracy: 0.6707317073170732


train epoch accuracy,▃▅▄▆▄▅▂▅▄▇▅▅▆▅█▄▇▅▁▃
train epoch loss,▇▄█▃▇▂▅▄▆▄▅▅▄▆█▄▁▄▇▄
valid accuracy,▄▅▁▂▆▃▂▆▆▇▅▅▅▅▅▅█▅▇▇
valid epoch loss,▆▆█▇▄▆▇▄▄▃▅▂▁▂▁▁▁▃▄▄
train epoch accuracy,0.58333
train epoch loss,0.08432
valid accuracy,0.62195
valid epoch loss,0.09078


In [23]:
##### Finetune Main Task #####
lr = 1e-4
num_epochs = 35

for param in model.parameters():
    param.requires_grad = True

data_augment = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
])

# data_augment = None
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(train_dataloader), epochs=num_epochs)
scheduler = None

wandb.init(project='182')
wandb.run.name = 'effnet_finetune'
model_save_name = wandb.run.name
train(model, optimizer=optimizer, data_augment=data_augment, scheduler=scheduler, train_dataloader=train_dataloader, validation_dataloader=validation_dataloader, criterion=criterion, num_epochs=num_epochs, save_name=model_save_name)
wandb.run.finish()

0.0001


100%|██████████| 33/33 [00:10<00:00,  3.03it/s]


Epoch 0 training loss:  0.0815667679364031


100%|██████████| 11/11 [00:00<00:00, 13.02it/s]


Validation loss:  0.09726313483424304
Validation acc:  0.4024390243902439
Saved checkpoint to model_ckpt/effnet_noaug.tar
Best valid accuracy: 0.4024390243902439


...

100%|██████████| 33/33 [00:10<00:00,  3.00it/s]


Epoch 34 training loss:  0.0036534434010250015


100%|██████████| 11/11 [00:00<00:00, 13.05it/s]


Validation loss:  0.11528771861297328
Validation acc:  0.7195121951219512
Best valid accuracy: 0.7804878048780488


train epoch accuracy,▁▅▇▇▇▇▇▇▇██████████████████████████
train epoch loss,█▆▄▃▂▂▂▂▂▁▂▂▂▂▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
valid accuracy,▁▆▅▇▆▆▆▇▇█▇▆▇▇▇▇▆█▇▇▆█▆▇█▇███▅▇▆▇▆▇
valid epoch loss,▃▃▄▂▂▄▅▅▃▁▃▃▄▅▃▄▆▄▆█▅▆█▅▃▄▄▅▆▇▇▆▅▅▅
train epoch accuracy,0.98864
train epoch loss,0.00365
valid accuracy,0.71951
valid epoch loss,0.11529


In [26]:
def test_model(model, test_dataloader, criterion):
    model.eval()
    with torch.no_grad():
        correct = 0
        for _, (inputs, labels) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
            inputs = inputs.float().to(device)
            labels = labels.to(device)
            pred = model(inputs)
            pred_labels = pred.argmax(1)
            correct += (pred_labels == labels).sum().item()
        acc = correct / len(test_dataloader.dataset)
        print("Test acc: ", acc)

In [28]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
model = models.efficientnet_b5()
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
load_checkpoint(model, save_name="effnet_noaug.tar")
model = model.to(device)
test_model(model, test_dataloader, nn.CrossEntropyLoss())

Loading checkpoint...
Done!


100%|██████████| 8/8 [00:00<00:00, 12.60it/s]

Test acc:  0.6666666666666666
